In [1]:
from MiniPAINN import MPAINNBlock, MPAINNPrediction
import torch
from torch.nn import Embedding, Module, Linear, SiLU, Embedding
from torch_geometric.nn import global_add_pool
from torch_geometric.nn.conv import MessagePassing
from train import mini_train
from utils import get_mini_dataloader, get_molecule, get_random_roto_reflection_translation, E3_transform_molecule, E3_transform_force, plot_molecules, plot_molecules_with_forces, bessel_rbf, cosine_cutoff

torch.manual_seed(2002)

In [7]:
def get_v_0(x):
    return x[:int(x.shape[0]*0.75)].view(-1,3,16)[0,:,:]

In [104]:
roto_reflection_translation = get_random_roto_reflection_translation()

molecule = get_molecule(type='benzene')
transformed_molecule = E3_transform_molecule(molecule=molecule, roto_reflection_translation=roto_reflection_translation)

Random roto-reflection:
	-0.144  0.833  0.534
	-0.548  0.382 -0.744
	 0.824  0.400 -0.401
Random translation:
	   0.0
	   0.0
	   0.0


/usr/local/Caskroom/miniconda/base/envs/GDL/lib/python3.11/site-packages/torch_geometric/data/dataset.py:239: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


In [105]:
model = Model()

In [106]:
print("---ORIGINAL---")
print(model(molecule))
print("---TRANSFORMED---")
print(model(transformed_molecule))

---ORIGINAL---
(tensor([[2.5635]], grad_fn=<SumBackward1>), tensor([[-0.0000,  0.2960, -0.0000],
        [ 0.2462,  0.1317, -0.0000],
        [ 0.2462, -0.1317, -0.0000],
        [-0.0000, -0.2960, -0.0000],
        [-0.2462, -0.1317, -0.0000],
        [-0.2462,  0.1317, -0.0000],
        [-0.0000, -0.5222, -0.0000],
        [-0.4496, -0.2603, -0.0000],
        [-0.4496,  0.2603, -0.0000],
        [-0.0000,  0.5222, -0.0000],
        [ 0.4496,  0.2603, -0.0000],
        [ 0.4496, -0.2603, -0.0000]]))
---TRANSFORMED---
(tensor([[2.5701]], grad_fn=<SumBackward1>), tensor([[ 0.2445,  0.1009,  0.1049],
        [ 0.0756, -0.0736,  0.2698],
        [-0.1453, -0.1930,  0.1366],
        [-0.2445, -0.1009, -0.1049],
        [-0.0756,  0.0736, -0.2698],
        [ 0.1453,  0.1930, -0.1366],
        [-0.4329, -0.1990, -0.2083],
        [-0.1525,  0.1481, -0.4757],
        [ 0.2808,  0.3446, -0.2665],
        [ 0.4329,  0.1990,  0.2083],
        [ 0.1525, -0.1481,  0.4757],
        [-0.2808, -0.344